# <center>PLANKTON CLASSIFICATION</center>
<center>Student: Akiyo Worou</center>

In [1]:
# importation of libraries
import torch
from torch import nn
import glob
import os
from torch.utils.data import Dataset, WeightedRandomSampler
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import csv

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

## Datasets, Preprocessing and Data loading

In [3]:
##
class SRDataset(Dataset):
    def __init__(self,path):
        self.folders_names = os.listdir(path)
        self.images_path=[]
        self.labels = []
        self.test_images_path = []
        self.test_labels = []
        for subfolder_name in self.folders_names:
            
            subfolder_images_names = glob.glob( os.path.join(path,subfolder_name , '*.jpg')   ) 
            subfolder_images_names_train, subfolder_images_names_test =train_test_split(subfolder_images_names , test_size = 0.2)
            
            ## train 
            self.images_path+= subfolder_images_names_train
            self.labels += [ int(subfolder_name[:3])  ]*len(subfolder_images_names_train) 
            
            ## test
            self.test_images_path +=subfolder_images_names_test
            self.test_labels += [ int(subfolder_name[:3])  ]*len(subfolder_images_names_test) 
            
    def __len__(self):
        return len(self.images_path)
    def __getitem__(self,index):
        label_ = torch.tensor(self.labels[index])
        im=torchvision.io.read_image(self.images_path[index])
        im=(1/255)*im
        im=transforms.Compose([
        #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.Resize((64,64),interpolation=transforms.InterpolationMode.BILINEAR)
        ])(im)
        
        return im,label_ 
    
    def get_test_set(self):
        return self.test_images_path,self.test_labels
    def get_distribution(self):
        return dict(Counter(self.labels))
    def classes(self):
        indices = [[] for i in range(len(self.get_distribution().keys()))]
        for i in range(len(self.labels)):
            indices[self.labels[i]].append(i)
        return indices
    def weights(self):
        weights_ = [0]*len(self.labels)
        labels_distribution = train_dataset.get_distribution()
        sum_val = sum(labels_distribution.values())
        for i in range(len(self.labels)):
            weights_[i] = sum_val/labels_distribution[ self.labels[i]]
        return weights_
        

      

In [4]:
class SREvalDataset(Dataset):
    def __init__(self,test_images_path,test_labels):
        self.test_images_path = test_images_path
        self.test_labels = test_labels
        
    def __len__(self):
        return len(self.test_images_path)
    def __getitem__(self,index):
        label_ = torch.tensor(self.test_labels[index])
        im=torchvision.io.read_image(self.test_images_path[index])
        im=(1/255)*im
        im=transforms.Compose([
        #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.Resize((64,64),interpolation=transforms.InterpolationMode.BILINEAR)
        ])(im)
        
        return im,label_ 
    def get_distribution(self):
        return dict(Counter(self.test_labels))
    def classes(self):
        indices = [[] for i in range(len(self.get_distribution().keys()))]
        for i in range(len(self.test_labels)):
            indices[self.test_labels[i]].append(i)
        return indices
    def weights(self):
        weights_ = [0]*len(self.test_labels)
        labels_distribution = train_dataset.get_distribution()
        sum_val = sum(labels_distribution.values())
        for i in range(len(self.test_labels)):
            weights_[i] = sum_val/labels_distribution[ self.test_labels[i]]
        return weights_

In [5]:
### batch_sampler finally not used

class Sampler():
    def __init__(self, classes):
        self.classes = classes

    def __iter__(self):
        batches = []
        classes = (self.classes).copy()
        classes_size = [len(el) for el in classes]
        m = min(classes_size)
        classes_size_norm = [ el//m for el in classes_size ]
        for i in range(m-1):
            batch = []
            for j in range(len(classes_size_norm)):
                for k in range(classes_size_norm[j]):
                    index = random.randint(0,len(classes[j])-1)
                    batch.append(classes[j].pop(index))
            batches.append(batch)
        batches.append(  [val for sublist in classes for val in sublist] )
        
            
        return iter(batches)

In [6]:

train_path='../input/plankton-dataset/train/train'
#dataset = SRDataset(train_path)

#dataset_size = len(dataset)
#train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [ int(dataset_size*0.7), dataset_size - int(dataset_size*0.7)])

train_dataset = SRDataset(train_path)

eval_dataset = SREvalDataset( *train_dataset.get_test_set() )



In [7]:
### the dataloader

def get_data_loader(dataset):

    ## get the labels distributions
    labels_distribution = dataset.get_distribution()

    ## weights for sampling
    weights =dataset.weights()
    sampler = WeightedRandomSampler(weights,len(weights))

    return torch.utils.data.dataloader.DataLoader(
        dataset,
        batch_size=64,
        sampler = sampler,
        drop_last=True,
        pin_memory=True)

In [8]:
train_dataloader = get_data_loader(train_dataset)
eval_dataloader = get_data_loader(eval_dataset)

In [9]:

print(f" * Dataset contains {len(train_dataset)} image(s).")
for _, batch in enumerate(train_dataloader, 0):
    image, label = batch
    #lr_image=lr_image[0, ...].mul(255).byte()
    #hr_image=hr_image[0, ...].mul(255).byte()
    #print(lr_image.shape)
    #print(lr_image[0, ...].mul(255).byte().shape)
    plt.imshow(image[0, ...].mul(255).byte().reshape(64,64),cmap='gray')
    #torchvision.io.write_png(image[0, ...].mul(255).byte(), "image.png")
    print(label)
    break # we deliberately break after one batch as this is just a test


## Create  Models


In [10]:
images_size = 64
nb_class = 86

In [11]:
# Model paper 

class convModel(torch.nn.Module):
    def __init__(self,interpolation='bilinear'):
        super().__init__()
            
        # First convolution 
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=images_size, kernel_size=3, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=3,stride = 2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=images_size, out_channels=2*images_size, kernel_size=3, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride = 2)
        
        # Convolution 3
        self.cnn3 = nn.Conv2d(in_channels=2*images_size, out_channels=2*images_size, kernel_size=3, stride=1, padding=0)
        self.relu3 = nn.ReLU()
        
        # Max pool 3
        self.maxpool3 = nn.MaxPool2d(kernel_size=2,stride = 2)
        
        # Convolution 4
        self.cnn4 = nn.Conv2d(in_channels=2*images_size, out_channels=4*images_size, kernel_size=3, stride=1, padding=0)
        self.relu4 = nn.ReLU()
        
        # Max pool 4
        self.maxpool4 = nn.MaxPool2d(kernel_size=2,stride = 2)
        
        # Convolution 5
        self.cnn5 = nn.Conv2d(in_channels=4*images_size, out_channels=4*images_size, kernel_size=3, stride=1, padding=0)
        self.relu5 = nn.ReLU()
        
        # Max pool 5
        self.maxpool5 = nn.MaxPool2d(kernel_size=2,stride = 2)
        
        
        # Fully connected 1
        self.fc1 = nn.Linear(256, 256) 
        
        ## 
        self.dropout1 =nn.Dropout(p=0.2)
        
        # Fully connected 2
        self.fc2 = nn.Linear(256, 256) 
        
        ## 
        self.dropout2 =nn.Dropout(p=0.2)
        
        self.output = nn.Linear(256, nb_class) 
        
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # flatten
        out = out.view(out.size(0), -1)
        
        # Linear function (readout)
        out = self.fc1(out)
        #print(out.shape)
        return out

In [12]:
# Model definition

class myModel(torch.nn.Module):
    def __init__(self,interpolation='bilinear'):
        super().__init__()
            
        # First convolution 
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=images_size//2, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=images_size//2, out_channels=images_size, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1
        self.fc1 = nn.Linear(images_size * 169, nb_class) 
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # flatten
        out = out.view(out.size(0), -1)
        
        # Linear function (readout)
        out = self.fc1(out)
        #print(out.shape)
        return out

In [13]:
# Model parameters check
#model=myModel()
model = convModel()
model.to(device)
num_params=0
for param in model.parameters():
    num_params+=param.numel()
print(num_params)

## Task 3 - Implement the Training Loop

In [14]:
### Not implemented yet
class F1_Loss(nn.Module):
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        print(y_pred.shape,y_true.shape)
        return f1_score(y_pred,y_true,average='macro')
        

In [15]:
# Some usefuls elements to setup
saving_path='myModel.pth'

number_of_epoch = 20


loss_function = torch.nn.CrossEntropyLoss() ## F1_Loss()
learning_rate=1e-4

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
    lr=learning_rate)

In [16]:
model =myModel()

# checkpoint = torch.load('../input/model-saved/myModel.pth')
# model.load_state_dict(checkpoint['model_state_dict'])

model.to(device)

In [17]:
def train_model(model,saving_path,learning_rate=0.0001,DEBUG  = False):
    
    Loss_list=[]
    eval_loss_list=[] 
    train_score_list = []
    eval_score_list =[]
    
    for epoch in range(number_of_epoch):
        Global_loss=0.0
        train_score = 0.0
        test_score = 0.0
        n_train = 0
        for images, labels in train_dataloader:
            images, labels = images.to(device), labels.to(device)
            # reset the gradient
            optimizer.zero_grad()
            # forward pass through the model
            labels_prediction = model(images)
            
            # compute the loss
            loss = loss_function(labels_prediction, labels)
            # adding up
            Global_loss+=loss.item()
            # backpropagation
            loss.backward()
            # update the model parameters
            optimizer.step()
            
            ## scoring
            train_score += f1_score ( torch.max(labels_prediction,1)[1].cpu().data.numpy(), labels.cpu().data.numpy(),average='macro' )
            n_train+=1
            if DEBUG:
                break
            # Evaluation on the test 
        with torch.no_grad():
            n=0
            eval_loss=0
            for  images_test,labels_test in eval_dataloader:
                images_test, labels_test = images_test.to(device), labels_test.to(device)
                # forward pass through the model
                labels_prediction = model(images_test)

                eval_loss+=loss_function(labels_prediction, labels_test).item()
                
                test_score += f1_score ( torch.max(labels_prediction,1)[1].cpu().data.numpy(), labels_test.cpu().data.numpy(),average='macro' )
                n+=1
                if DEBUG:
                    break
                
            eval_loss_list.append(eval_loss/n)
            eval_score_list.append(test_score/n)
            
        
         # log the metrics, images, etc
        Global_loss/=n_train
        Loss_list.append(Global_loss)
        train_score_list.append(train_score/n_train)
        print('epoch : ', epoch +1 ,'---> train loss : ',Global_loss ,' ---> eval loss : ',eval_loss/n , ' ---> train score : ',train_score/n_train, ' ---> test score : ',test_score/n )
        
        # saving state
        if not DEBUG:
            torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_score':  train_score_list,
                    'test_score' :  eval_score_list,
                    'eval_loss': eval_loss_list,
                    'loss': Loss_list}, saving_path)
    
    fig = plt.figure(figsize=(20,15))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    ax2 = fig.add_subplot(2, 2, 1)
    ax2.plot(Loss_list)
    ax2.set_title('Train Loss')

    ax3 = fig.add_subplot(2, 2, 2)
    ax3.plot(eval_loss_list)
    ax3.set_title('Eval Loss')

    ax4 = fig.add_subplot(2, 2, 3)
    ax4.plot(train_score_list)
    ax4.set_title('Train score')

    ax5 = fig.add_subplot(2, 2, 4)
    ax5.plot( eval_loss_list )
    ax5.set_title('Eval Score')

    plt.show()

In [ ]:
train_model(model,saving_path,DEBUG=False)

In [ ]:
class SRTestDataset(Dataset):
    def __init__(self,test_path):
        self.test_images_path = glob.glob( os.path.join(test_path , '*.jpg')   )
    def __len__(self):
        return len(self.test_images_path)
    def __getitem__(self,index):
        im_name = self.test_images_path[index].split('/')[-1]
        im=torchvision.io.read_image(self.test_images_path[index])
        im=(1/255)*im
        im=transforms.Compose([
        #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.Resize((64,64),interpolation=transforms.InterpolationMode.BILINEAR)
        ])(im)
        
        return im,im_name

In [ ]:
## predictions

def make_predictions(test_path,DEBUG=False):
    test_dataset = SRTestDataset(test_path)
    test_predictions ={ 'imgname': [], 'label':[]}
    
    test_dataloader = torch.utils.data.dataloader.DataLoader(
        test_dataset,
        batch_size=64)
    with torch.no_grad():
        for  images_test,images_name in test_dataloader:
            images_test = images_test.to(device)
            labels_prediction = torch.max(model(images_test),1)[1].cpu().data.numpy()
                
            for i in range(len(images_name)):
                test_predictions['label'].append(labels_prediction[i])
                test_predictions['imgname'].append(images_name[i])
                if DEBUG:
                    print( images_name[i], labels_prediction[i])
            if DEBUG:
                break
    ### saving the result 
    with open("predictions.csv", "w") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(test_predictions.keys())
        writer.writerows(zip(*test_predictions.values()))
        


In [ ]:
test_path = '../input/plankton-dataset/test/test/imgs' 
make_predictions(test_path,DEBUG=False)